# mosi_audio_and_AED

2 étapes clés dans ce notebook :  

1) Extraction des features audio .pt
- À partir des fichiers .wav, on calcule des MFCC (13 coefficients sur 50 frames).
- Ces MFCC sont directement stockés dans des fichiers .pt.
- Ces fichiers .pt servent de features audio numériques dans DEVANet.

2) Extraction AED (Audio Emotional Description)
- On calcule 4 indicateurs émotionnels : pitch, loudness, jitter, shimmer.
- On génère une phrase décrivant la manière de parler (ex : “The speaker uses a soft voice with low pitch…”).
- Cette phrase est encodée par BERT → produit D_a (embedding 128d).


In [ ]:
import os
import librosa
import torch
import numpy as np

# Dossiers
input_dir = "MOSI_full/audio"  # fichiers .wav
output_dir = "MOSI_full/features_audio"       # sauvegarde des vecteurs audio
os.makedirs(output_dir, exist_ok=True)

# Paramètres
n_mfcc = 13        # nombre de coefficients MFCC
max_len = 50       # nombre maximum de frames pour DEVA
sr_target = 16000  # fréquence d'échantillonnage

def extract_mfcc(filepath, n_mfcc=n_mfcc, max_len=max_len):
    """
    Extrait les MFCC d'un fichier audio et fait padding/troncature à max_len frames.
    """
    y, sr = librosa.load(filepath, sr=sr_target)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # shape (n_mfcc, T)
    mfcc = mfcc.T  # shape (T, n_mfcc)

    # Padding ou troncature pour avoir max_len frames
    if mfcc.shape[0] < max_len:
        pad = np.zeros((max_len - mfcc.shape[0], n_mfcc))
        mfcc = np.vstack([mfcc, pad])
    else:
        mfcc = mfcc[:max_len]

    return torch.tensor(mfcc, dtype=torch.float)

# Boucle sur tous les fichiers .wav
for f in os.listdir(input_dir):
    if f.endswith(".wav"):
        vec = extract_mfcc(os.path.join(input_dir, f))
        out_file = os.path.join(output_dir, f.replace(".wav", ".pt"))
        torch.save(vec, out_file)
        print("Saved:", out_file)


In [ ]:
import os
import librosa
import torch
import numpy as np

from deva import TextEncoder   

# Dossiers
input_dir = "MOSI_full/audio"          # fichiers .wav
output_dir = "MOSI_full/AED_features"  # sauvegarde AED
os.makedirs(output_dir, exist_ok=True)

# --------------------------------------------------
# AUDIO EMOTIONAL FEATURES (Pitch, Loudness, Jitter, Shimmer)
# --------------------------------------------------

def extract_audio_features(filepath, sr_target=16000):
    """
    Extrait :
    - pitch (pyin)
    - loudness (rms)
    - jitter (variation relative du pitch)
    - shimmer (variation d'amplitude)
    Version simplifiée mais suffisante pour AED DEVA.
    """
    y, sr = librosa.load(filepath, sr=sr_target)

    # Pitch
    f0, voiced_flag, _ = librosa.pyin(
        y,
        fmin=50, fmax=300,
        sr=sr_target,
        frame_length=1024
    )
    pitch = np.nanmean(f0)

    # Loudness = RMS
    loudness = librosa.feature.rms(y=y).mean()

    # Jitter = variation du pitch
    jitter = np.nanmean(np.abs(np.diff(f0))) / (pitch + 1e-5)

    # Shimmer = variation d'amplitude
    shimmer = np.mean(np.abs(np.diff(np.abs(y))))

    return pitch, loudness, jitter, shimmer


# --------------------------------------------------
# PHRASE TEXTUELLE EMOTIONNELLE
# --------------------------------------------------

def audio_description(pitch, loudness, jitter, shimmer):
    """
    Convertit les valeurs audio en phrase descriptive anglaise.
    """
    pitch_desc = "low pitch" if pitch < 140 else "high pitch"
    loud_desc = "soft voice" if loudness < 0.02 else "strong voice"
    jitter_desc = "stable tone" if jitter < 0.01 else "unstable tone"
    shimmer_desc = "smooth amplitude" if shimmer < 0.005 else "irregular amplitude"

    sentence = (
        f"The speaker uses a {pitch_desc}, with a {loud_desc}, "
        f"a {jitter_desc}, and a {shimmer_desc}."
    )
    return sentence


# --------------------------------------------------
# ENCODAGE AED -> D_a via TextEncoder DEVA
# --------------------------------------------------

# Instanciation du TextEncoder DEVA
text_encoder = TextEncoder(bert_pretrained="bert-base-uncased")

def encode_AED_sentence(sentence):
    """
    Encode la phrase émotionnelle audio en D_a = [1, 8, 128]
    via le TextEncoder du modèle DEVA.
    """

    tokenizer = text_encoder.bert.tokenizer

    encoded = tokenizer(sentence, return_tensors="pt",
                        padding="max_length", truncation=True, max_length=50)

    bert_input = torch.stack([
        encoded["input_ids"],
        encoded["attention_mask"],
        encoded["token_type_ids"]
    ], dim=1)  # shape [1, 3, L]

    with torch.no_grad():
        D_a = text_encoder(bert_input)  # [1, 8, 128]

    return D_a


# --------------------------------------------------
# BOUCLE PRINCIPALE : AUDIO -> AED -> D_a
# --------------------------------------------------

for f in os.listdir(input_dir):

    if f.endswith(".wav"):
        filepath = os.path.join(input_dir, f)

        # 1. EXTRACTION DES FEATURES EMOTIONNELS
        pitch, loudness, jitter, shimmer = extract_audio_features(filepath)

        # 2. CONVERSION EN PHRASE
        sentence = audio_description(pitch, loudness, jitter, shimmer)

        # 3. ENCODAGE PAR TEXTENCODER
        D_a = encode_AED_sentence(sentence)

        # 4. SAUVEGARDE
        out_file = os.path.join(output_dir, f.replace(".wav", "_AED.pt"))
        torch.save(D_a, out_file)

        print("Saved AED:", out_file)
